In [1]:
from scipy import stats
srcc = stats.spearmanr( [0.6480761, 0.62233955, 0.6661059, 0.631339, 0.6192408, 0.641552, 0.6423639, 0.7225227, 0.65772206, 0.63077545, 0.54655486  , 0.6763791, 0.62823856, 0.63159883, 0.62590545, 0.65006614, 0.6068853, 0.63621056, 0.5723472, 0.6442189, 0.62065214, 0.67560524, 0.6549503, 0.61165047], [0.062, 0.032, 0.029, 0.028, 0.027, 0.104, 0.028, 0.006, 0.033, 0.088, 0.052, 0.014, 0.029, 0.032, 0.022, 0.0, 0.012, 0.04, 0.051, 0.003, 0.032, 0.031, 0.019, 0.042])
srcc[0]

-0.3369615411989142

In [2]:
%cd ..


d:\quality_assessment\knowledge_distillation\ckdn\code\knowledge-distillation-master


In [3]:
import torch
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())


NVIDIA GeForce GTX 1050
True


In [4]:
import predict_one_image



In [5]:
iqa = predict_one_image.IQA_CKDN()

In [6]:
iqa.predict(restored_addr="tst_imgs/i01_19_4.bmp" , degraded_addr= "tst_imgs/i01_23_1.bmp")

0.5934968

In [7]:
dte = iqa.get_DTE_features("tst_imgs/i01_19_4.bmp")

In [8]:
dte[0].shape

(256,)

# vgg features


In [9]:
import torch
from torchvision import models, transforms
from torch.autograd import Variable

# Load the pretrained model
model = models.vgg16(pretrained=True)

# Use the model object to select the desired layer
block4 = model.features[:24]

# Set the model to evaluation mode
block4.eval()

c:\Users\Farhad\.conda\envs\gpu\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Farhad\.conda\envs\gpu\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [10]:
from PIL import Image


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

block4 = block4.to(device)
block4.eval()

transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the image
image = Image.open("tst_imgs/i01_19_4.bmp")

# Apply the transformations and add an extra dimension for the batch
image = transform(image).unsqueeze(0)

# Ensure we're doing this on CPU
image = image.to(device)

# Forward pass
output = block4(image)
output = torch.nn.functional.adaptive_max_pool2d(output, (1, 1))
# Print output

In [11]:
import torch
from torchvision import models, transforms
from torch.autograd import Variable
from PIL import Image

# Load the pretrained model
model = models.vgg16(pretrained=True)

# Use the model object to select the desired layer
block4 = model.features[:24]



def get_features_from_vgg_block_4(img_addr , block4 = block4):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    block4 = block4.to(device)
    block4.eval()

    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # Load the image
    image = Image.open(img_addr)

    # Apply the transformations and add an extra dimension for the batch
    image = transform(image).unsqueeze(0)

    # Ensure we're doing this on CPU
    image = image.to(device)

    # Forward pass
    output = block4(image)
    output = torch.nn.functional.adaptive_max_pool2d(output, (1, 1))

    output = torch.squeeze(output)

    # Convert the tensor to a numpy array
    output_np = output.cpu().detach().numpy()
    return output_np


len(get_features_from_vgg_block_4("tst_imgs/i01_19_4.bmp" , block4 = block4))

512

## correlation or cosine distance

In [12]:
from scipy import stats
import numpy as np
from scipy.spatial.distance import cosine


r = "tst_imgs/i01_23_1.bmp"
d = "tst_imgs/i01_19_4.bmp"


rr = iqa.get_DTE_features(r)[0]
dd = iqa.get_DTE_features(d)[0]

srcc = stats.spearmanr( rr , dd)
srcc[0] , cosine(rr, dd)

(0.9977554924451721, 0.0005650520324707031)

## cosine on TID in vgg feature space

In [14]:

contents = ["01","02","03","04","05","06","07","08","09" , "10", "11", "12", "13", "14", "15", "16", "17", "18", "19","20","21","22","23","24","25"]


import glob
TID_ADDR = "D:\quality_assessment\DS/tid2013"

In [19]:
def get_refs_content(content):
        return glob.glob(TID_ADDR+'/reference_images/*'+content+'*') * 120

In [20]:
def get_images_content(im_name):
    imgs = glob.glob(TID_ADDR+'/distorted_images/*'+im_name+'_'+ '*' +'_'+ '*' +'.*')
    return imgs

In [ ]:


row = 3


for c in contents:
    imgs = get_images_content(c)
    refs = get_refs_content(c)
    for i in range(len(imgs)):
        rr = get_features_from_vgg_block_4(refs[i])
        dd = get_features_from_vgg_block_4(imgs[i])
        

        # worksheet.write(row, i+4, cosine(rr, dd))
        worksheet.write(row, i+4,  cosine(rr, dd))

    
    print(row)
    row +=1

workbook.close()    

# normalize by other refs

In [24]:
import xlsxwriter
 
workbook = xlsxwriter.Workbook('COSINE_on_VGG_features_normalize_by_other_refs.xlsx')
worksheet = workbook.add_worksheet()

In [25]:
import random

row = 3

for c in contents:
    rand_ref = random.sample(contents, 1)[0]
    while rand_ref == c:
       rand_ref = random.sample(contents, 1)[0]
     
    imgs = get_images_content(c)
    refs = get_refs_content(c)
    other_refs = get_refs_content(rand_ref)
    for i in range(len(imgs)):
        dd = get_features_from_vgg_block_4(imgs[i])
        rr = get_features_from_vgg_block_4(refs[i])
        other_rr = get_features_from_vgg_block_4(other_refs[i])
        
        cos_ref = cosine(rr, dd)
        cos_other_ref = cosine(other_rr, dd)
        # worksheet.write(row, i+4, cosine(rr, dd))
        worksheet.write(row, i+4,  cos_ref / cos_other_ref)

    
    print(row)
    row +=1

workbook.close()     



  

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


# Correlation


# dst:

In [15]:
import glob
dsts = ["01","02","03","04","05","06","07","08","09" , "10", "11", "12", "13", "14", "15", "16", "17", "18", "19","20","21","22","23","24"]


def get_images_dst(dst_type):
    imgs = glob.glob(TID_ADDR+'/distorted_images/*_'+ dst_type +'_'+ '*' +'.*')
    return imgs

def get_mos_dst(dst_type ):
    mos = []
    f = open(TID_ADDR+"/mos_with_names.txt", "r")
    lines = f.readlines()
    for l in lines:
        if l.split(' ')[1].split('_')[1] == dst_type:
            mos.append(float(l.split(' ')[0][:5]))

    return mos


refs = glob.glob(TID_ADDR+'/reference_images/*')

refs_for_dst = []

for elem in refs:
    for i in range(5):
        refs_for_dst.append(elem)

In [16]:
import xlsxwriter
 
workbook = xlsxwriter.Workbook('SRCC_on_VGG-cosine_and_mos.xlsx')
worksheet = workbook.add_worksheet()


row = 3


for c in dsts:
    imgs = get_images_dst(c)
    refs = refs_for_dst
    cosines = []
    moses = get_mos_dst(c)
    for i in range(len(imgs)):
        rr = get_features_from_vgg_block_4(refs[i])
        dd = get_features_from_vgg_block_4(imgs[i])
        
        cosines.append(cosine( rr , dd))

    worksheet.write(row, 3,  c)
    worksheet.write(row, 4,  stats.spearmanr( cosines , moses)[0])

    
    print(row)
    row +=1

workbook.close()  

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


all imgs


In [17]:
imgs = []
refs = []
moses = []
for c in dsts:
    imgs += get_images_dst(c)
    refs += refs_for_dst
    moses += get_mos_dst(c)


cosines = []
for i in range(len(imgs)):
    rr = get_features_from_vgg_block_4(refs[i])
    dd = get_features_from_vgg_block_4(imgs[i])
    
    cosines.append(cosine( rr , dd))


print(stats.spearmanr( cosines , moses)[0])

-0.8444548058342226


content

In [18]:
contents = ["01","02","03","04","05","06","07","08","09" , "10", "11", "12", "13", "14", "15", "16", "17", "18", "19","20","21","22","23","24","25"]


def get_images_content(im_name):
    imgs = glob.glob(TID_ADDR+'/distorted_images/*'+im_name+'_'+ '*' +'_'+ '*' +'.*')
    return imgs

def get_mos_content(im_name ):
    mos = []
    f = open(TID_ADDR+"/mos_with_names.txt", "r")
    lines = f.readlines()
    for l in lines:
        if l.split(' ')[1][1:3] == im_name:
            mos.append(float(l.split(' ')[0][:5]))

    return mos


def get_refs_content(content):
        return glob.glob(TID_ADDR+'/reference_images/*'+content+'*') * 120





import xlsxwriter
 
workbook = xlsxwriter.Workbook('SRCC_on_VGG-cosine_and_mos-content.xlsx')
worksheet = workbook.add_worksheet()



row = 3


for c in contents:
    imgs = get_images_content(c)
    refs = get_refs_content(c)
    cosines = []
    moses = get_mos_content(c)
    for i in range(len(imgs)):
        rr = get_features_from_vgg_block_4(refs[i])
        dd = get_features_from_vgg_block_4(imgs[i])
        
        cosines.append(cosine( rr , dd))

    worksheet.write(row, 3,  c)
    worksheet.write(row, 4,  stats.spearmanr( cosines , moses)[0])

    
    print(row)
    row +=1

workbook.close() 

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27


all imgs

In [19]:
imgs = []
refs = []
moses = []
for c in contents:
    imgs += get_images_content(c)
    refs += get_refs_content(c)
    moses += get_mos_content(c)


cosines = []
for i in range(len(imgs)):
    rr = get_features_from_vgg_block_4(refs[i])
    dd = get_features_from_vgg_block_4(imgs[i])
    
    cosines.append(cosine( rr , dd))

stats.spearmanr( cosines , moses)[0]

-0.8444548058342226